In [1]:
import math
import time
import wandb
import spam
import pprint

In [2]:
#spam.mlp_classifier((784, 30, 10), epochs=1)

In [14]:
#spam.reduce_precision(1.3, 8)
dir(spam)

<module 'spam' from 'C:\\Users\\Lubko\\AppData\\Roaming\\Python\\Python39\\site-packages\\spam.cp39-win_amd64.pyd'>

In [4]:
from distutils.sysconfig import get_python_lib
print(get_python_lib())

c:\users\lubko\appdata\local\programs\python\python39\Lib\site-packages


In [5]:
#import importlib
#importlib.reload(spam)

In [2]:
wandb.login()
#wandb.init(project="my-test-project")

wandb: Currently logged in as: lubomirkurcak (use `wandb login --relogin` to force relogin)


True

In [3]:
# vytvorenie sweep configu (pouzivame nahodne zvolenie parametrov)
sweep_config = {
    'method': 'random'
}

In [4]:
#metric = {
#    'name': 'loss',
#    'goal': 'minimize'
#}

metric = {
    'name': 'correct_classifications',
    'goal': 'maximize'
}

sweep_config['metric'] = metric

In [5]:
# diskretne parametre
parameters_dict = {
    'fp_subnormal': {
        'values': ['CPFLOAT_SUBN_RND',
                   'CPFLOAT_SUBN_USE']
    },
    
    'fp_round': {
        'values': ['CPFLOAT_RND_NA',
                   'CPFLOAT_RND_NZ',
                   'CPFLOAT_RND_NE',
                   'CPFLOAT_RND_TP',
                   'CPFLOAT_RND_TN',
                   'CPFLOAT_RND_TZ',
                   'CPFLOAT_RND_SP',
                   'CPFLOAT_RND_SE',
                   'CPFLOAT_RND_OD',
                   'CPFLOAT_NO_RND']
    },
    
    'fp_flip': {
        'values': ['CPFLOAT_NO_SOFTERR',
                   'CPFLOAT_SOFTERR']
    },
    
    'fp_explim': {
        'values': ['CPFLOAT_EXPRANGE_STOR',
                   'CPFLOAT_EXPRANGE_TARG']
    },
}

sweep_config['parameters'] = parameters_dict

In [6]:
# konstantne parametre
parameters_dict.update({
    'layer_sizes': {
        'value': (784, 30, 10)
    },
    'epochs': {
        'value': 5
    },
    'minibatch_size': {
        'value': 10
    },
    'learning_rate': {
        'value': 0.1
    },
    'weight_decay': {
        'value': 0.0001
    },
    'dropout': {
        'value': 0
    },
    'momentum_coefficient': {
        'value': 0
    },
    'loss': {
        'value': 'mean_squared_error'
    }
})

In [7]:
'''
precision
   * @brief Bits of precision of target format.
   *
   * @details The maximum values allowed are 24 and 53 if the storage format is
   * `float` or `double`, respectively.
'''

'''
maximum exponent
   * @brief Maximum exponent of target format.
   *
   * @details The maximum values allowed are 127 and 1023 if the storage format
   * is `float` or `double`, respectively. Larger values are reduced to the
   * maximum allowed value without warning. This field is ignored unless
   * `explim` is set to `CPFLOAT_EXPRANGE_TARG`.   
'''

# spojite parametre
parameters_dict.update({
    'fp_p': {
        'distribution': 'uniform',
        'min': 0,
        'max': 0.5
    },
    
    'fp_precision': {
        'distribution': 'q_uniform',
        'q': 1,
        'min': 4,
        'max': 24,
    },
    
    'fp_emax': {
        'distribution': 'q_uniform',
        'q': 1,
        'min': 4,
        'max': 127,
    },
})


'''
'fp_precision': {
    'distribution': 'q_log_uniform',
    'q': 1,
    'min': math.log(4),
    'max': math.log(24),
},

'fp_emax': {
    'distribution': 'q_log_uniform',
    'q': 1,
    'min': math.log(4),
    'max': math.log(127),
},
'''

"\n'fp_precision': {\n    'distribution': 'q_log_uniform',\n    'q': 1,\n    'min': math.log(4),\n    'max': math.log(24),\n},\n\n'fp_emax': {\n    'distribution': 'q_log_uniform',\n    'q': 1,\n    'min': math.log(4),\n    'max': math.log(127),\n},\n"

In [8]:
pprint.pprint(sweep_config)

{'method': 'random',
 'metric': {'goal': 'maximize', 'name': 'correct_classifications'},
 'parameters': {'dropout': {'value': 0},
                'epochs': {'value': 5},
                'fp_emax': {'distribution': 'q_uniform',
                            'max': 127,
                            'min': 4,
                            'q': 1},
                'fp_explim': {'values': ['CPFLOAT_EXPRANGE_STOR',
                                         'CPFLOAT_EXPRANGE_TARG']},
                'fp_flip': {'values': ['CPFLOAT_NO_SOFTERR',
                                       'CPFLOAT_SOFTERR']},
                'fp_p': {'distribution': 'uniform', 'max': 0.5, 'min': 0},
                'fp_precision': {'distribution': 'q_uniform',
                                 'max': 24,
                                 'min': 4,
                                 'q': 1},
                'fp_round': {'values': ['CPFLOAT_RND_NA',
                                        'CPFLOAT_RND_NZ',
                     

In [9]:
spam.system('echo %TIME%')

0

In [10]:
# inicializuj sweep pomocou konfigu (vytvori sa stranka sweepu, kde je mozne monitorovat behy a analyzovat parametre)
sweep_id = wandb.sweep(sweep_config, project="my-test-project")

Create sweep with ID: p304s2tf
Sweep URL: https://wandb.ai/lubomirkurcak/my-test-project/sweeps/p304s2tf


In [11]:
# trenovacia funkcia, pouzita pri volani trenovacich agentov
def train():
    
    # inicialuzuj wandb a vyber config s nastavenymi hodnotami podla sweep configu
    run = wandb.init()
    config = run.config
    
    '''
    for dummy_epoch in range(30):
        wandb.log({"loss": 0.9 ** dummy_epoch,
                   "accuracy": 1 - 0.97 ** dummy_epoch}, commit=False, step=dummy_epoch)
    '''
    
    correct_classifications = spam.mlp_classifier(tuple(config["layer_sizes"]),
        epochs=config["epochs"],
        minibatch_size=config["minibatch_size"],
        learning_rate=config["learning_rate"],
        weight_decay=config["weight_decay"],
        dropout=config["dropout"],
        momentum_coefficient=config["momentum_coefficient"],
        fp_precision=config["fp_precision"],
        fp_emax=config["fp_emax"],
        fp_subnormal=config["fp_subnormal"],
        fp_round=config["fp_round"],
        fp_flip=config["fp_flip"],
        fp_p=config["fp_p"],
        fp_explim=config["fp_explim"]
    )
    
    
    wandb.log({'correct_classifications': correct_classifications}, commit=False, step=0)
    
    '''
    print('Early accuracy stop!')
    wandb.alert(
        title='Early accuracy stop!', 
        text=f'Accuracy {correct/total} is above 0.7',
        level=AlertLevel.INFO, # INFO, WARN, or ERROR 
        wait_duration=timedelta(minutes=1)
    )
    '''
        
    run.finish()
    

In [12]:
#spam.create_forward_net([784, 30, 10])
#spam.create_forward_net((784, 30, 10))

In [ ]:
#wandb.agent("i91zir54", project="my-test-project", function=train, count=400)

wandb: Agent Starting Run: bl8qvbgq with config:
wandb: 	dropout: 0
wandb: 	epochs: 5
wandb: 	fp_emax: 103
wandb: 	fp_explim: CPFLOAT_EXPRANGE_TARG
wandb: 	fp_flip: CPFLOAT_SOFTERR
wandb: 	fp_p: 0.14981953639934964
wandb: 	fp_precision: 24
wandb: 	fp_round: CPFLOAT_RND_NE
wandb: 	fp_subnormal: CPFLOAT_SUBN_USE
wandb: 	layer_sizes: [784, 30, 10]
wandb: 	learning_rate: 0.1
wandb: 	loss: mean_squared_error
wandb: 	minibatch_size: 10
wandb: 	momentum_coefficient: 0
wandb: 	weight_decay: 0.0001
wandb: wandb version 0.12.8 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


cpfloat_validate_optstruct returned 0
Epoch 0/5 Batch 0/6000 Cost 3.567867 LR 0.100000
Epoch 0/5 Batch 1024/6000 Cost 0.445373 LR 0.100000
Epoch 0/5 Batch 2048/6000 Cost 0.102295 LR 0.100000
Epoch 0/5 Batch 3072/6000 Cost 0.361814 LR 0.100000
Epoch 0/5 Batch 4096/6000 Cost 0.234994 LR 0.100000
Epoch 0/5 Batch 5120/6000 Cost 0.258456 LR 0.100000
Epoch 0/5 Correct 9096/10000 Best 9096(0)Epoch 1/5 Batch 0/6000 Cost 0.073508 LR 0.100000
Epoch 1/5 Batch 1024/6000 Cost 0.302620 LR 0.100000
Epoch 1/5 Batch 2048/6000 Cost 0.083255 LR 0.100000
Epoch 1/5 Batch 3072/6000 Cost 0.336903 LR 0.100000
Epoch 1/5 Batch 4096/6000 Cost 0.196128 LR 0.100000
Epoch 1/5 Batch 5120/6000 Cost 0.225061 LR 0.100000
Epoch 1/5 Correct 9166/10000 Best 9166(0)Epoch 2/5 Batch 0/6000 Cost 0.062918 LR 0.100000
Epoch 2/5 Batch 1024/6000 Cost 0.304445 LR 0.100000
Epoch 2/5 Batch 2048/6000 Cost 0.067025 LR 0.100000
Epoch 2/5 Batch 3072/6000 Cost 0.312913 LR 0.100000
Epoch 2/5 Batch 4096/6000 Cost 0.168790 LR 0.100000
Epoch

wandb: Network error (SSLError), entering retry loop. See W:\shared\python\gymrl\wandb\run-20211216_193709-bl8qvbgq\logs\debug-internal.log for full traceback.


Epoch 4/5 Batch 5120/6000 Cost 0.184596 LR 0.100000
Epoch 4/5 Correct 9264/10000 Best 9264(0)

correct_classifications,9264
_runtime,738
_timestamp,1639680567
_step,0


...

cpfloat_validate_optstruct returned 0
Epoch 0/5 Batch 0/6000 Cost 3.755166 LR 0.100000
Epoch 0/5 Batch 1024/6000 Cost 0.887000 LR 0.100000
Epoch 0/5 Batch 2048/6000 Cost 0.922997 LR 0.100000
Epoch 0/5 Batch 3072/6000 Cost 0.895755 LR 0.100000
Epoch 0/5 Batch 4096/6000 Cost 0.913445 LR 0.100000
Epoch 0/5 Batch 5120/6000 Cost 0.868835 LR 0.100000
Epoch 0/5 Correct 958/10000 Best 958(0)Early stopping due to poor performance


correct_classifications,958
_runtime,103
_timestamp,1639875139
_step,0


correct_classifications,▁
_runtime,▁
_timestamp,▁
_step,▁


wandb: Agent Starting Run: e720d1hn with config:
wandb: 	dropout: 0
wandb: 	epochs: 5
wandb: 	fp_emax: 31
wandb: 	fp_explim: CPFLOAT_EXPRANGE_STOR
wandb: 	fp_flip: CPFLOAT_SOFTERR
wandb: 	fp_p: 0.14722990853247447
wandb: 	fp_precision: 7
wandb: 	fp_round: CPFLOAT_RND_NA
wandb: 	fp_subnormal: CPFLOAT_SUBN_RND
wandb: 	layer_sizes: [784, 30, 10]
wandb: 	learning_rate: 0.1
wandb: 	loss: mean_squared_error
wandb: 	minibatch_size: 10
wandb: 	momentum_coefficient: 0
wandb: 	weight_decay: 0.0001
wandb: wandb version 0.12.9 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


cpfloat_validate_optstruct returned 0
Epoch 0/5 Batch 0/6000 Cost 1.788866 LR 0.100000
Epoch 0/5 Batch 1024/6000 Cost 0.359275 LR 0.100000
Epoch 0/5 Batch 2048/6000 Cost 0.100950 LR 0.100000
Epoch 0/5 Batch 3072/6000 Cost 0.358566 LR 0.100000
Epoch 0/5 Batch 4096/6000 Cost 0.243961 LR 0.100000


In [13]:
wandb.agent(sweep_id, train, count=4)

wandb: Agent Starting Run: ham2iphr with config:
wandb: 	dropout: 0
wandb: 	epochs: 5
wandb: 	fp_emax: 13
wandb: 	fp_explim: CPFLOAT_EXPRANGE_STOR
wandb: 	fp_flip: CPFLOAT_NO_SOFTERR
wandb: 	fp_p: 0.4569428214002166
wandb: 	fp_precision: 12
wandb: 	fp_round: CPFLOAT_RND_NE
wandb: 	fp_subnormal: CPFLOAT_SUBN_USE
wandb: 	layer_sizes: [784, 30, 10]
wandb: 	learning_rate: 0.1
wandb: 	loss: mean_squared_error
wandb: 	minibatch_size: 10
wandb: 	momentum_coefficient: 0
wandb: 	weight_decay: 0.0001
wandb: wandb version 0.12.9 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


cpfloat_validate_optstruct returned 0
Epoch 0/5 Batch 0/6000 Cost 2.982569 LR 0.100000
Epoch 0/5 Batch 1024/6000 Cost 0.388711 LR 0.100000
Epoch 0/5 Batch 2048/6000 Cost 0.135067 LR 0.100000
Epoch 0/5 Batch 3072/6000 Cost 0.354132 LR 0.100000
Epoch 0/5 Batch 4096/6000 Cost 0.373939 LR 0.100000
Epoch 0/5 Batch 5120/6000 Cost 0.232934 LR 0.100000
Epoch 0/5 Correct 8890/10000 Best 8890(0)Epoch 1/5 Batch 0/6000 Cost 0.106049 LR 0.100000
Epoch 1/5 Batch 1024/6000 Cost 0.432708 LR 0.100000
Epoch 1/5 Batch 2048/6000 Cost 0.118648 LR 0.100000
Epoch 1/5 Batch 3072/6000 Cost 0.341228 LR 0.100000
Epoch 1/5 Batch 4096/6000 Cost 0.310484 LR 0.100000
Epoch 1/5 Batch 5120/6000 Cost 0.197761 LR 0.100000
Epoch 1/5 Correct 9008/10000 Best 9008(0)Epoch 2/5 Batch 0/6000 Cost 0.102055 LR 0.100000
Epoch 2/5 Batch 1024/6000 Cost 0.417651 LR 0.100000
Epoch 2/5 Batch 2048/6000 Cost 0.088061 LR 0.100000
Epoch 2/5 Batch 3072/6000 Cost 0.298117 LR 0.100000
Epoch 2/5 Batch 4096/6000 Cost 0.266143 LR 0.100000
Epoch

wandb: Network error (ConnectionError), entering retry loop. See W:\rpnn\python\wandb\run-20220103_001123-ham2iphr\logs\debug-internal.log for full traceback.
Error while writing to file :write to closed file


correct_classifications,9148
_runtime,268
_timestamp,1641165351
_step,0


correct_classifications,▁
_runtime,▁
_timestamp,▁
_step,▁


...

wandb: Network error (SSLError), entering retry loop. See W:\rpnn\python\wandb\run-20220103_003808-f5qcmxry\logs\debug-internal.log for full traceback.


Epoch 3/5 Correct 9215/10000 Best 9215(0)Epoch 4/5 Batch 0/6000 Cost 0.076390 LR 0.100000
Epoch 4/5 Batch 1024/6000 Cost 0.374637 LR 0.100000
Epoch 4/5 Batch 2048/6000 Cost 0.061181 LR 0.100000
Epoch 4/5 Batch 3072/6000 Cost 0.241663 LR 0.100000
Epoch 4/5 Batch 4096/6000 Cost 0.153349 LR 0.100000
Epoch 4/5 Batch 5120/6000 Cost 0.124314 LR 0.100000
Epoch 4/5 Correct 9268/10000 Best 9268(0)

wandb: Network error resolved after 0:10:54.465603, resuming normal operation.


correct_classifications,9268
_runtime,679
_timestamp,1641167367
_step,0


correct_classifications,▁
_runtime,▁
_timestamp,▁
_step,▁


In [18]:
'''
import gym
env = gym.make('CartPole-v0')

for i_episode in range(20):
    observation = env.reset()
    for t in range(100):
        env.render()
        #print(observation)
        #action = env.action_space.sample()
        #action = (t+1)//2%2
        action = 1 if observation[2] > 0 else 0

        observation, reward, done, info = env.step(action)
        if done:
            print("Episode finished after {} timesteps".format(t+1))
            break
env.close()
'''

'\nimport gym\nenv = gym.make(\'CartPole-v0\')\n\nfor i_episode in range(20):\n    observation = env.reset()\n    for t in range(100):\n        env.render()\n        #print(observation)\n        #action = env.action_space.sample()\n        #action = (t+1)//2%2\n        action = 1 if observation[2] > 0 else 0\n\n        observation, reward, done, info = env.step(action)\n        if done:\n            print("Episode finished after {} timesteps".format(t+1))\n            break\nenv.close()\n'

wandb: Network error resolved after 0:07:53.465766, resuming normal operation.
